## [Deep Neural Networks are Surprisingly Reversible](https://arxiv.org/abs/2107.06304)

Define variables for input/output paths based on environment

In [ ]:
# from google.colab import drive
# from pathlib import Path

# drive.mount('/content/drive')
# DRIVE_PATH = Path('/content/drive/My Drive/ML_Training_Data/inverse-of-neural-net')

In [ ]:
from pathlib import Path

INPUT_PATH = Path('../inputs/mnist/cnn')
OUTPUT_PATH = Path('../outputs/mnist/cnn')

# if training on colab with Drive storage
# INPUT_PATH = DRIVE_PATH / 'inputs/mnist/cnn'
# OUTPUT_PATH = DRIVE_PATH / 'outputs/mnist/cnn'

INPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

Define the "target" model which we want to reverse/invert. Each layer used in this model will have a corresponding layer in the inverted model, which should perform the "reverse" of what target model layer does. Some layers like MaxPool layers are tricky, need to try those next. For now, I have skipped pooling here.

The purpose of the paper was to be able to find a reverse of a target Neural Net so that the inputs on which it was trained on can be recovered (check the paper for motivation behind it), assuming that we don't have access to original dataset.

That's why here we will train the model on MNIST dataset, but train the inverted model on QMNIST. But in the paper, authors have used a sophisticated way to generated small subset of input images with same statistics are original imputs. But, for this small demo I will use existing MNIST-like datasets already generated by researchers.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from collections import OrderedDict
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(999)


class Net(nn.Module):
    def __init__(self, in_dim, in_channel, out_size):
        super(Net, self).__init__()

        # 4 layers config
        layer1 = self.get_conv_leakyrelu(in_channel, 5, 2, stride=2, slope=0.2)
        layer2 = self.get_conv_leakyrelu(5, 10, 2, stride=1, slope=0.2)
        layer1_out = layer1(torch.rand((1, 1, 28, 28)))
        layer2_out = layer2(layer1_out)
        print(f'conv outs={layer1_out.size()}, {layer2_out.size()}')
        layer3 = self.get_linear_with_leakyrelu(layer2_out.flatten(1).size(-1), int(layer2_out.flatten(1).size(-1)/2), 0.2)
        layer4 = nn.Linear(int(layer2_out.flatten(1).size(-1)/2), out_size)
        self.layers = nn.ModuleList([layer1, layer2, layer3, layer4])
        
        # 3 layers config
        # layer1 = self.get_conv_relu(in_channel, 20, 3)
        # layer1_out = layer1(torch.rand((1, 1, 28, 28)))
        # print(f'conv outs={layer1_out.size()}')
        # layer2 = self.get_linear_with_relu(layer1_out.flatten(1).size(-1), int(layer1_out.flatten(1).size(-1)/2))
        # layer3 = nn.Linear(int(layer1_out.flatten(1).size(-1)/2), out_size)
        # self.layers = nn.ModuleList([layer1, layer2, layer3])

        self.forward_vals = []
        
    def forward(self, x):
        self.forward_vals.clear()
        for layer in self.layers:
            _ = layer[0] if isinstance(layer, nn.Sequential) else layer
            if isinstance(_, nn.Linear) and len(x.size()) > 2:
                x = x.flatten(1)
            x = layer(x)
            self.forward_vals.append(x)
        return x
    
    def get_conv_relu_maxpool(self, in_channel, num_filters, kernel_size, pool_size):
        return nn.Sequential(nn.Conv2d(in_channel, num_filters, kernel_size),
                             nn.MaxPool2d(pool_size),
                             nn.ReLU())
        
    def get_conv_relu(self, in_channel, num_filters, kernel_size):
        return nn.Sequential(nn.Conv2d(in_channel, num_filters, kernel_size),
                             nn.ReLU())
        
    def get_conv_leakyrelu(self, in_channel, num_filters, kernel_size, stride=1, slope=0.2):
        return nn.Sequential(nn.Conv2d(in_channel, num_filters, kernel_size, stride=stride),
                             nn.LeakyReLU(slope))

    def get_linear_with_relu(self, inp, out):
        return nn.Sequential(nn.Linear(inp, out), nn.ReLU())
    
    def get_linear_with_leakyrelu(self, inp, out, neg_slope):
        return nn.Sequential(nn.Linear(inp, out), nn.LeakyReLU(neg_slope))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128
train_kwargs = {'batch_size': batch_size,
               'shuffle': True}
test_kwargs = {'batch_size': batch_size,
              'shuffle': True}

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

dataset1 = datasets.MNIST(INPUT_PATH/'mnist', train=True, download=True,
                   transform=transform)
dataset2 = datasets.MNIST(INPUT_PATH/'mnist', train=False,
                   transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net(28, 1, 10).to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)

model, model(torch.rand(2, 1, 28, 28).to(device)).size()

In [ ]:
from matplotlib import pyplot as plt

examples = iter(test_loader)
example_data, example_targets = examples.next()
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
plt.show()

### Target Model Training

If you have not already trained a model, uncomment the next cell and run it.

In [ ]:
# import copy
# import warnings
# warnings.filterwarnings("ignore") 

# def train(model, device, train_loader, optimizer, epoch):
#     model.train()
#     for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
#         # print(data.size(), data.view(data.size(0), -1).size())
#         optimizer.zero_grad()
#         output = model(data)
#         loss = F.cross_entropy(output, target)
#         loss.backward()
#         optimizer.step()
#         if batch_idx % 100 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, batch_idx * len(data), len(train_loader.dataset),
#                 100. * batch_idx / len(train_loader), loss.item()))
            

# def test():
#     with torch.no_grad():
#         n_correct = 0
#         n_samples = 0
#         for images, labels in test_loader:
#             images = images.to(device)
#             labels = labels.to(device)
#             outputs = model(images)
#             # max returns (value ,index)
#             _, predicted = torch.max(outputs.data, 1)
#             n_samples += labels.size(0)
#             n_correct += (predicted == labels).sum().item()

#     acc = 100.0 * n_correct / n_samples
#     print(f'Accuracy of the network on the 10000 test images: {acc} %')
#     return acc


# best_acc = -1
# best_model = None
# for epoch in range(1, 50):
#     train(model, device, train_loader, optimizer, epoch)
#     acc = test()
#     if acc > best_acc:
#         best_acc = acc
#         best_model = copy.deepcopy(model)
#         print(f'new best acc={best_acc}')
#     else:
#         print(f'current acc={acc}, prev_best_acc={best_acc}')

# state = {
#     'model_state': best_model.state_dict(),
#     'test_acc': best_acc
# }
# torch.save(state, OUTPUT_PATH/'state_4layer_noPool_nn.pt')
# print('best model saved')
# # 98.88

### Reverse Model Section

Let's first load the saved best target model.

In [ ]:
state = torch.load(OUTPUT_PATH/'state_4layer_noPool_nn.pt')
state.keys(), state['test_acc']

In [ ]:
model.load_state_dict(state['model_state'])
model.to(device)

for param in model.parameters():
    param.requires_grad = False
                                 
model

Configurations which are used in the current experiment/run

In [ ]:
config_dict = {
    'epochs': 600,
    'alpha': 1,
    'lr': 1e-3,
    'lr_sched': 'exp',
    'gamma': 0.99,
    'loss': 'abs',
    'min_lr': 1e-5,
    'max_lr': 1e-4,
    'leaky_relu': 0.2,
    'actv': 'leaky_relu',
    'use_bn': False,
    'batch_size': 128,
    'init_method': 'lecun_normal',
    'optim': 'adam',
    'momentum': 0.9
}

If you're using a different dataset, you may want to normalize it with appropriate stats. For MNIST I have used constant figure from internet directly (which is close to mean and std already). In the code, the usage of get_stats() is commented out to show how it can be used.

In [ ]:
def get_stats():
    train_qmnist = datasets.QMNIST(INPUT_PATH/'qmnist', train = True, download= True,
                             transform=transforms.ToTensor())
    dataloader = torch.utils.data.DataLoader(train_qmnist, batch_size=128)
    nimages = 0
    mean = 0.0
    var = 0.0
    for i_batch, batch_target in enumerate(dataloader):
        batch = batch_target[0]
        # Rearrange batch to be the shape of [B, C, W * H]
        batch = batch.view(batch.size(0), batch.size(1), -1)
        # Update total number of images
        nimages += batch.size(0)
        # Compute mean and std here
        mean += batch.mean(2).sum(0) 
        var += batch.var(2).sum(0)

    mean /= nimages
    var /= nimages
    std = torch.sqrt(var)
    return mean, std

In [ ]:
model(torch.rand((1, 1, 28, 28)).to(device))
[print(val.size()) for val in model.forward_vals]

In [ ]:
from torch.nn.init import xavier_normal_, xavier_uniform_
from torch.nn.init import zeros_, uniform_, _calculate_correct_fan, normal_
from torch.nn.init import kaiming_uniform_, kaiming_normal_

class InvNet(nn.Module):
    def __init__(self, out_dim, out_channel, tgt_model):
        super(InvNet, self).__init__()
        
        layers = []
        for i, tgt_layer in enumerate(tgt_model.layers[::-1], 1):
            if isinstance(tgt_layer, nn.Sequential):
                tgt_layer = tgt_layer[0]
            
            if isinstance(tgt_layer, nn.Linear):
                if len(tgt_model.forward_vals[-i-1].size()) > 2:
                    out_size = tgt_model.forward_vals[-i-1].flatten(1).size(-1)
                    self.first_conv_dim = tgt_model.forward_vals[-i-1].size()
                else:
                    out_size = tgt_model.forward_vals[-i-1].size(-1)
                
                # use this var to only use activations based on some conditions, here always True
                use_actv = True
                layer = self.get_linear_with_relu(tgt_model.forward_vals[-i].size(-1), out_size, config_dict['use_bn'], use_actv)
            elif isinstance(tgt_layer, nn.Conv2d):
                if i < len(tgt_model.layers):
                    out_channel = tgt_model.forward_vals[-i-1].size(1)
                    layer = self.get_conv_relu(tgt_model.forward_vals[-i].size(1), out_channel, 2, 1, 1, 1, 
                                               True if len(config_dict['actv'])>0 else False,
                                              config_dict['use_bn'])
                else:
                    # last layer
                    out_channel = 1
                    layer = self.get_upsample_and_conv(tgt_model.forward_vals[-i].size(1), out_channel, 2, 1, 0, 1, False, False)
                
            layers.append(layer)

        self.conv_type = type(layers[-2][0])
        self.layers = nn.ModuleList(layers)
        self.forward_vals = []
        
    def forward(self, x):
        self.forward_vals.clear()
        for i, layer in enumerate(self.layers):
            if isinstance(layer[0], self.conv_type) and len(x.size()) <= 2:
                x = x.view(x.size(0), self.first_conv_dim[1], self.first_conv_dim[2], self.first_conv_dim[3])  # out dims of conv2d of target model
            x = layer(x)
            self.forward_vals.append(x)
        return x
    
    def get_conv_relu(self, in_channel, num_filters, kernel_size, stride, padding, dilation, use_actv=True, use_bn=False):
        sequential = nn.Sequential(nn.Conv2d(in_channel, num_filters, kernel_size, 
                                                    stride=stride, padding=padding, dilation=dilation,
                                                    bias=True))
        
        if use_bn:
            sequential.add_module('bn', nn.BatchNorm2d(num_filters))
        
        if use_actv:
            sequential.add_module('actv', self.get_activation(config_dict['actv']))
            
        return sequential

    def get_upsample_and_conv(self, in_channel, num_filters, kernel_size, stride, padding, dilation, use_actv=True, use_bn=False):
        sequential = nn.Sequential(nn.Upsample(size=29, mode='bilinear'),
            nn.Conv2d(in_channel, num_filters, kernel_size,
                                                    stride=stride, padding=padding, dilation=dilation,
                                                    bias=True)
            )
        
        if use_bn:
            sequential.add_module('bn', nn.BatchNorm2d(num_filters))
        
        if use_actv:
            sequential.add_module('actv', self.get_activation(config_dict['actv']))
            
        return sequential

    def get_conv_and_upsample(self, in_channel, num_filters, kernel_size, stride, padding, dilation, use_actv=True, use_bn=False):
        sequential = nn.Sequential(nn.Conv2d(in_channel, num_filters, kernel_size,
                                            stride=stride, padding=padding, dilation=dilation,
                                            bias=True),
                                   nn.Upsample(size=28, mode='bilinear')
                                   )
        
        if use_bn:
            sequential.add_module('bn', nn.BatchNorm2d(num_filters))
        
        if use_actv:
            sequential.add_module('actv', self.get_activation(config_dict['actv']))
            
        return sequential

    def get_activation(self, actv):
        if actv == 'leaky_relu':
            return nn.LeakyReLU(config_dict['leaky_relu'])
        elif actv == 'relu':
            return nn.ReLU()
        elif actv == 'selu':
            return nn.SELU()
        elif actv == 'tanh':
            return nn.Tanh()
    
    def get_linear_with_relu(self, inp, out, use_bn, use_actv=True):
        sequential =  nn.Sequential(nn.Linear(inp, out))
        if use_bn:
            sequential.add_module('lin_bn', nn.BatchNorm1d(out))
        if use_actv:
            sequential.add_module('lin_actv', self.get_activation(config_dict['actv']))
        return sequential


def lecun_normal_(weight, mode):
    fan = _calculate_correct_fan(weight, mode)
    limit = torch.sqrt(3/torch.tensor(fan))
    normal_(weight, -limit, limit)


def init_weights(layer):
    def init_(data):
        if config_dict['init_method'] == 'he_normal':
            kaiming_normal_(data, a=config_dict['leaky_relu'], nonlinearity='selu')
        elif config_dict['init_method'] == 'xavier_normal':
            xavier_normal_(data)
        elif config_dict['init_method'] == 'lecun_normal':
            lecun_normal_(data, 'fan_in') 

    if isinstance(layer, nn.Linear):
        init_(layer.weight)
        # zeros_(layer.bias)
    elif isinstance(layer, nn.Conv2d):
        init_(layer.weight)
        # zeros_(layer.bias)


inv_model = InvNet(28, 1, model)
inv_model.apply(init_weights)
inv_model.to(device)
# mean, std = get_stats()
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) # or use mean, std here
        ])

batch_size = config_dict['batch_size']
test_kwargs = {'batch_size': batch_size,
              'shuffle': True}
train_qmnist = datasets.QMNIST(INPUT_PATH/'qmnist', train = True, download= True,
                             transform=transform) 
test_qmnist = datasets.QMNIST(INPUT_PATH/'qmnist', train = False, download= True,
                             transform=transform)

print(f'size of inv data={train_qmnist}')
inv_data_loader = torch.utils.data.DataLoader(train_qmnist, **test_kwargs)

if config_dict['loss'] == 'mse':
    loss_fn = nn.MSELoss()
elif config_dict['loss'] == 'huber':
    loss_fn = nn.HuberLoss()
else:
    loss_fn = nn.L1Loss()

if config_dict['optim'] == 'adam':
    optimizer = optim.Adam(inv_model.parameters(), lr=config_dict['lr'])
elif config_dict['optim'] == 'sgd':
    optimizer = optim.SGD(inv_model.parameters(), lr=config_dict['lr'],
                          momentum=config_dict['momentum'], nesterov=True)
elif config_dict['optim'] == 'nadam':
    optimizer = optim.NAdam(inv_model.parameters(), lr=config_dict['lr'])

lr_sched = None
if config_dict['lr_sched'] == 'exp':
    lr_sched = optim.lr_scheduler.ExponentialLR(optimizer, config_dict['gamma'], verbose=False)

print(inv_model,'\n', loss_fn, lr_sched)
o = inv_model(torch.rand((2, 10)).to(device))
print(f'out_size={o.size()}')
[val.size() for val in inv_model.forward_vals], inv_model.first_conv_dim

In [ ]:
from matplotlib import pyplot as plt
import random

for i in range(6):
    example_data, example_target = test_qmnist[random.randint(0, len(test_qmnist))]
    ax = plt.subplot(2,3,i+1)
    ax.set_title(f'label={example_target}')
    plt.imshow(example_data.squeeze(), cmap='gray')
plt.tight_layout()
plt.show()

In [ ]:
def get_layer_map(model):
    layer_map = {}
    for i in range(len(model.layers)):
        layer_map[i] = len(model.layers)-(i+1)
    print(f'layer_map={layer_map}')
    return layer_map

In [ ]:
def layer_wise_loss(model, inv_model, inp, k=-1):
    loss = 0
    for layer_num, layer in enumerate(model.layers[:k+1] if k > -1 else model.layers):
        tgt_layer_in = inp if layer_num == 0 else model.forward_vals[layer_num-1]  # get the input tensor for this layer in tgt model
        tgt_layer_out = model.forward_vals[layer_num]  # get the output of i-th target model layer 
        inv_layer = inv_model.layers[tgt_to_inv_layer_map[layer_num]]  # get the corresponding layer in reverse model
        inv_layer_out = inv_layer(tgt_layer_out)  # feed the target model layer output to reverse model layer
        if inv_layer_out.size(-1) != tgt_layer_in.size(-1):
            tgt_layer_in = tgt_layer_in.flatten(1)
        loss += loss_fn(inv_layer_out, tgt_layer_in)  # outputs of both layer should be similar
        
    return loss


def layer_wise_loss_progressive(model, inv_model, inp, layer_num):
    tgt_layer_in = inp if layer_num == 0 else model.forward_vals[layer_num-1]  # get the input tensor for this layer in tgt model
    tgt_layer_out = model.forward_vals[layer_num]  # get the output of i-th target model layer 
    inv_layer = inv_model.layers[tgt_to_inv_layer_map[layer_num]]  # get the corresponding layer in reverse model
    inv_layer_out = inv_layer(tgt_layer_out)  # feed the target model layer output to reverse model layer
    if inv_layer_out.size(-1) != tgt_layer_in.size(-1):
        tgt_layer_in = tgt_layer_in.flatten(1)
    return loss_fn(inv_layer_out, tgt_layer_in)  # outputs of both layer should be similar


def k_layer_loss(model, inv_model, inp, k=-1):
    loss = 0
    for layer_num, layer_ in enumerate(model.layers[:k+1] if k > -1 else model.layers):
        tgt_layer_out = model.forward_vals[layer_num]
        x = tgt_layer_out
        for layer in inv_model.layers[-(1+layer_num):]:
            if isinstance(layer[0], inv_model.conv_type) and len(x.size()) < 4:
                x = x.view(x.size(0), inv_model.first_conv_dim[1], inv_model.first_conv_dim[2], inv_model.first_conv_dim[3])
            x = layer(x)
        loss += loss_fn(x, inp)
    
    return loss


def k_layer_loss_progressive(model, inv_model, inp, layer_num):
    tgt_layer_out = model.forward_vals[layer_num]
    x = tgt_layer_out
    for layer in inv_model.layers[-(1+layer_num):]:
        if isinstance(layer[0], inv_model.conv_type) and len(x.size()) < 4:
            x = x.view(x.size(0), inv_model.first_conv_dim[1], inv_model.first_conv_dim[2], inv_model.first_conv_dim[3])
        x = layer(x)
    return loss_fn(x, inp)


def cycle_consistency_inversion_loss(model, inp, inv_out, k=-1):
    org_activations = copy.deepcopy(model.forward_vals[:k+1] if k > -1 else model.forward_vals)
    out_for_inv_input = model(inv_out)  # pass the input image generated by inverted network
    inv_activations = model.forward_vals[:k+1] if k > -1 else model.forward_vals
    loss = 0
    for org_actv, actv in zip(org_activations, inv_activations):
        loss += loss_fn(actv, org_actv)
        
    return loss

### all loss together

In [ ]:
import copy

tgt_to_inv_layer_map = get_layer_map(model)

def train_inv(model, inv_model, device, data_loader, optimizer, epoch, alpha, layer_wise_only=False, layer_num=-1):
    model.train()
    
    total_loss = 0
    total_loss_layer = 0
    total_loss_img = 0
    total_loss_cyc = 0
    
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
    
        out = model(data)  
        inv_out = inv_model(out)
        
        # step 1- layer wise load
        loss_layer = layer_wise_loss(model, inv_model, data)
        
        if not layer_wise_only:
            # step 2- upto layer-k loss
            loss_img = k_layer_loss(model, inv_model, data)

            # step 3- full network output loss
            loss_cyc = cycle_consistency_inversion_loss(model, data, inv_out)
        else:
            loss_img = torch.tensor(0)
            loss_cyc = torch.tensor(0)
        
        loss = loss_layer + loss_img + (alpha*loss_cyc)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_loss_layer += loss_layer.item()
        total_loss_img += loss_img.item()
        total_loss_cyc += loss_cyc.item()
        
    return total_loss / (batch_idx+1), total_loss_layer / (batch_idx+1), \
            total_loss_img / (batch_idx+1), total_loss_cyc / (batch_idx+1)

### Progressive optimization of each layer

In [ ]:
tgt_to_inv_layer_map = get_layer_map(model)

def train_inv_progressive(model, inv_model, device, data_loader, optimizer, epoch, alpha, layer_wise_only=False):
    model.train()
    
    total_loss = 0
    total_loss_layer = 0
    total_loss_img = 0
    total_loss_cyc = 0
    
    for layer_num in range(len(model.layers)):
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            out = model(data)
            org_activations = copy.deepcopy(model.forward_vals) 
            inv_out = inv_model(out)
            out_for_inv = model(inv_out)

            loss = layer_wise_loss_progressive(model, inv_model, data, layer_num) + \
                    k_layer_loss_progressive(model, inv_model, data, layer_num) + \
                    (alpha * loss_fn(model.forward_vals[layer_num], org_activations[layer_num]))
            
            loss.backward()
            optimizer.step()
        
        # this means I am not training inv_model conv2d with these losses!! is it an issue?
        # also, does "fine-tune" mean, after training a inv_layer k, all layers 1..k are taken as single module and optimized again using L_k?s
        if layer_num > 0:
            for batch_idx, (data, target) in enumerate(data_loader):
                data, target = data.to(device), target.to(device)
                optimizer.zero_grad()

                out = model(data)  
                inv_out = inv_model(out)

                loss_layer = layer_wise_loss(model, inv_model, data, layer_num)
                loss_img = k_layer_loss(model, inv_model, data, layer_num)
                loss_cyc = cycle_consistency_inversion_loss(model, data, inv_out, layer_num)

                loss = loss_layer + loss_img + (alpha*loss_cyc)

                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                total_loss_layer += loss_layer.item()
                total_loss_img += loss_img.item()
                total_loss_cyc += loss_cyc.item()
    
    div = (batch_idx+1)*(len(inv_model.layers)-1)
    return total_loss / div, total_loss_layer / div, \
            total_loss_img / div, total_loss_cyc / div

In [ ]:
run_id = 'temp'# 'newProgressive_noPoolInTgt_4layer_nonProgsv'
change_desc = 'non progressive training, tgt_model 4 layered lrelu without maxPool, inv_model with lrelu & upsample+conv'
is_resume = True
run_path = OUTPUT_PATH/'runs'/run_id
state_path = run_path/'final_state_4layer_noTgtPool_inv_nn.pt'
best_model_path = run_path/'state_4layer_noTgtPool_inv_nn.pt'

# used to log progress. not mandatory, just need to comment out "wandb" if not needed
# !pip install wandb

### Reverse Model Training

In [ ]:
import time
import sys
import wandb
import os
import numpy as np


def round_(n, d=5):
    return np.round(n, d)


run_path.mkdir(parents=True, exist_ok=True)

EPOCHS = config_dict['epochs']
ALPHA = config_dict['alpha']
start_epoch = 0
prev_best_loss = 999999
total_training_time = 0

if is_resume and state_path.exists():
    state = torch.load(state_path)
    inv_model.load_state_dict(state['model_state'])
    optimizer.load_state_dict(state['optimizer'])
    if lr_sched:
        lr_sched.load_state_dict(state['lr_sched'])
    start_epoch = state['epoch']+1
    prev_best_loss = state['prev_best_loss']

    # uncomment, if you want to override the saved previous lr
    # for g in optimizer.param_groups:
    #     g['lr'] = config_dict['lr']

    print(f'resuming training for {run_id} from epoch {start_epoch}, last loss {state["loss"]} prev_best_loss {prev_best_loss}')
    

wandb.init(project='nn-are-reversible',
           entity='nayash', save_code=True, id=run_id,
           name=run_id, notes=change_desc,
           dir=run_path,
           resume='allow',
           config=config_dict)
wandb.watch(inv_model)
wandb.log({'model_arch': str(inv_model)})

if run_id in ['test', 'temp']:
    os.environ['WANDB_MODE'] = 'offline'
    print(f'run_id={run_id}, so wandb is disabled!')
          
for epoch in range(start_epoch, EPOCHS):
    stime = time.time()
    total_loss, total_loss_layer, total_loss_img, total_loss_cyc = \
        train_inv(model, inv_model, device, inv_data_loader, optimizer, epoch, ALPHA)

    epoch_time = time.time() - stime
    total_training_time += epoch_time
    
    print(f'epoch={epoch}/{EPOCHS} loss={round_(total_loss)}, loss_layer={round_(total_loss_layer)}, loss_img={round_(total_loss_img)}, loss_cyc={round_(total_loss_cyc)}, epoch_duration={round(epoch_time)} secs')
    
    wandb.log({
        'loss':total_loss,
        'loss_layer':total_loss_layer,
        'loss_img':total_loss_img,
        'loss_cyc':total_loss_cyc,
        'lr': optimizer.param_groups[0]['lr']
    })
    
    if lr_sched is not None and optimizer.param_groups[0]['lr'] > config_dict['min_lr']:
        lr_sched.step()
    
    if total_loss < prev_best_loss:
        prev_best_loss = total_loss
        state = {
            'model_state': inv_model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'lr_sched': lr_sched.state_dict() if lr_sched else None,
            'epoch': epoch,
            'loss': prev_best_loss,
            'prev_best_loss': prev_best_loss
        }
        torch.save(state, best_model_path)
        print(f'new best loss={total_loss}')

        
state = {
    'model_state': inv_model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_sched': lr_sched.state_dict() if lr_sched else None,
    'epoch': epoch,
    'loss': total_loss,
    'prev_best_loss': prev_best_loss
    }
torch.save(state, state_path)
        
print(f'{EPOCHS} epochs finished in {total_training_time/60} mins')
wandb.finish()

Run this cell, if the above "training" cell was stopped half way to save the states

In [ ]:
state = {
    'model_state': inv_model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_sched': lr_sched.state_dict() if lr_sched else None,
    'epoch': epoch,
    'loss': total_loss,
    'prev_best_loss': prev_best_loss
    }
torch.save(state, state_path)
wandb.finish()

Plot and compare the original vs generated images (1st and 2nd row). 3rd and 4th row plot the outputs for any target intermediate layer and the input for the corresponding inverted layer, to validate that the activations are acutally similar.

In [ ]:
import torch
import time
import numpy as np

state = torch.load(best_model_path)
inv_model.load_state_dict(state['model_state'])

plt.figure(figsize=(15, 10))
total_plts = 10

for i in range(total_plts):
    _ = np.random.randint(len(test_qmnist))
    inp = model(test_qmnist[_][0].unsqueeze(0).to(device))
    o = inv_model(inp)
    img = o.reshape((28, 28)).cpu().detach().numpy()
    ax_org = plt.subplot(4, 10, i+1)
    plt.imshow(test_qmnist[_][0].squeeze().numpy(), cmap='gray')
    ax_pred = plt.subplot(4, 10, i+1+total_plts)
    plt.imshow(img, cmap='gray')
    label = torch.argmax(model(o))
    ax_pred.set_title(f'pred={label}')
    ax_org.set_title(f'tgt={test_qmnist[_][1]}')
    
    ax_tgt_fmap = plt.subplot(4, 10, i+1+2*total_plts)
    # view the outputs of first conv layer
    plt.imshow(model.forward_vals[1].squeeze().mean(0).cpu().detach().numpy(), cmap='gray')
    ax_inv_fmap = plt.subplot(4, 10, i+1+3*total_plts)
    # view the inputs to last conv layer of inverted model. It should be close to first tgt conv layers output
    plt.imshow(inv_model.forward_vals[1].view(tuple(inv_model.first_conv_dim)).squeeze().mean(0).cpu().detach().numpy(), cmap='gray')
    ax_tgt_fmap.set_title('tgt_fmap')
    ax_inv_fmap.set_title('inv_fmap')

plt.tight_layout()
sample_save_path = run_path/f'{run_id}_{time.time()}.png'
plt.savefig(sample_save_path)
plt.show()

Let's see if target model is able to classify properly the generated images too

In [ ]:
from tqdm.notebook import tqdm

tgts = []
preds = []
with torch.no_grad():
    for i, test_sample in enumerate(tqdm(test_qmnist)):
        inp = model(test_qmnist[i][0].unsqueeze(0).to(device))
        tgt = test_qmnist[i][1]
        o = inv_model(inp)  # generate image using inverted model
        pred = torch.argmax(torch.softmax(model(o), -1))
        tgts.append(tgt)
        preds.append(pred.item())

print(f'accuracy on generated images = {(np.array(tgts) == np.array(preds)).sum()/len(tgts) * 100}%')

Also, let's see how does the images generated on the training set look.

In [ ]:
import torch
import time
import numpy as np

state = torch.load(best_model_path)
inv_model.load_state_dict(state['model_state'])

plt.figure(figsize=(15, 5))
total_plts = 10
for i in range(total_plts):
    _ = np.random.randint(len(train_qmnist))
    inp = model(train_qmnist[_][0].unsqueeze(0).to(device))
    o = inv_model(inp)
    img = o.reshape((28, 28)).cpu().detach().numpy()
    ax_org = plt.subplot(2, 10, i+1)
    plt.imshow(train_qmnist[_][0].squeeze().numpy(), cmap='gray')
    ax_pred = plt.subplot(2, 10, i+1+total_plts) 
    plt.imshow(img, cmap='gray')
    label = torch.argmax(model(o))
    ax_pred.set_title(f'pred={label}')
    ax_org.set_title(f'tgt={train_qmnist[_][1]}')
plt.tight_layout()
sample_save_path = run_path/f'{run_id}_trainSetSamples_{time.time()}.png'
plt.savefig(sample_save_path)
plt.show()